# 高级得深度学习最佳实践

## 不用Sequential模型的解决方案：Keras函数是API

Sequrntial模型假设网络只有一个输入和一个输出，而且网络是层的线性堆叠，这是一个经过普遍验证的假设。

有些网络需要多个独立的输入，有些网络则需要多个输出，而有些网络在层与层之间具有内部分支，这使得网络看起来像是层构成的**图(graph)**，而不是线性堆叠。

### 函数式API简介

In [2]:
from keras import Input, layers

input_tensor = Input(shape=(32,))
dense = layers.Dense(32,activation='relu')

output_tensor = dense(input_tensor)

Using TensorFlow backend.


In [3]:
from keras.models import Sequential, Model
from keras import layers
from keras import Input

# Sequential 实现
seq_model = Sequential()
seq_model.add(layers.Dense(32,activation='relu',input_shape=(64,)))
seq_model.add(layers.Dense(32,activation='relu'))
seq_model.add(layers.Dense(10,activation='softmax'))

# API实现
x = layers.Dense(32,activation='relu')(input_tensor)
x = layers.Dense(32,activation='relu')(x)
output_tensor = layers.Dense(10,activation='softmax')(x)

model = Model(input_tensor, output_tensor)

model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_6 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_7 (Dense)              (None, 10)                330       
Total params: 2,442
Trainable params: 2,442
Non-trainable params: 0
_________________________________________________________________


对这种Model实例进行编译、训练或评估时，其API与Sequential模型相同。

In [5]:
model.compile(optimizer='rmsprop',loss='categorical_crossentropy')

import numpy as np
x_train = np.random.random((1000,32))
y_train = np.random.random((1000,10))

model.fit(x_train,y_train,epochs=10,batch_size=128)

score = model.evaluate(x_train,y_train)

Epoch 1/10
1000/1000 [==============================] - 2s 2ms/step - loss: 11.6783
Epoch 2/10
1000/1000 [==============================] - 0s 20us/step - loss: 12.0335
Epoch 3/10
1000/1000 [==============================] - 0s 26us/step - loss: 12.7626
Epoch 4/10
1000/1000 [==============================] - 0s 19us/step - loss: 14.0101
Epoch 5/10
1000/1000 [==============================] - 0s 18us/step - loss: 15.4453
Epoch 6/10
1000/1000 [==============================] - 0s 19us/step - loss: 16.8621
Epoch 7/10
1000/1000 [==============================] - 0s 17us/step - loss: 18.3077
Epoch 8/10
1000/1000 [==============================] - 0s 17us/step - loss: 19.9407
Epoch 9/10
1000/1000 [==============================] - 0s 18us/step - loss: 21.5687
Epoch 10/10
1000/1000 [==============================] - 0s 48us/step


### 多输入模型
函数式API可用于构建具有多个输入的模型。通常情况下，这种模型会在某一时刻用一个组合多个张量的层将不同的输入分支合并，张量组合方式可能是相加、连接等。

In [12]:
from keras.models import Model
from keras import layers
from keras import Input

text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 10000

text_input = Input(shape=(None,),dtype='int32',name='text')

embedding_text = layers.Embedding(
    text_vocabulary_size,64)(text_input)

encoded_text = layers.LSTM(32)(embedding_text)

question_input = Input(shape=(None,),
                       dtype='int32',
                       name='question')

embedded_question = layers.Embedding(
    question_vocabulary_size,32)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)

concatenated = layers.concatenate([encoded_text,encoded_question],
                                   axis=-1)

answer = layers.Dense(answer_voacbulary_size,
                      activation='softmax')(concatenated)

model = Model([text_input,question_input],answer)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])

In [14]:
import keras
import numpy as np

num_samples = 1000
max_length = 100

text = np.random.randint(1,text_vocabulary_size,
                         size=(num_samples,max_length))
question = np.random.randint(1,question_vocabulary_size,
                             size=(num_samples,max_length))
answers = np.random.randint(answer_vocabulary_size, size=
                            (num_samples))

answers = keras.utils.to_categorical(answers,
                                     answer_vocabulary_size)

model.fit([text,question],answers,epochs=10,batch_size=128)

model.fit({'text':text,'question':question},answers,
          epochs=10,batch_size=128)

D:\ProgramFile\Anaconda\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
1000/1000 [==============================] - 2s 2ms/step - loss: 9.2103 - acc: 0.0010
Epoch 2/10
1000/1000 [==============================] - 1s 943us/step - loss: 9.1552 - acc: 0.0050
Epoch 3/10
1000/1000 [==============================] - 1s 939us/step - loss: 8.6704 - acc: 0.0000e+00
Epoch 4/10
1000/1000 [==============================] - 1s 915us/step - loss: 8.1842 - acc: 0.0030
Epoch 5/10
1000/1000 [==============================] - 1s 928us/step - loss: 7.8394 - acc: 0.0030
Epoch 6/10
1000/1000 [==============================] - 1s 923us/step - loss: 7.5869 - acc: 0.0030
Epoch 7/10
1000/1000 [==============================] - 1s 911us/step - loss: 7.4034 - acc: 0.0030
Epoch 8/10
1000/1000 [==============================] - 1s 911us/step - loss: 7.2716 - acc: 0.0030
Epoch 9/10
1000/1000 [==============================] - 1s 947us/step - loss: 7.1768 - acc: 0.0030
Epoch 10/10
1000/1000 [==============================] - 1s 935us/step - loss: 7.1080 - acc: 0.0030
Epoch 1

### 多输出模型
![多输出模型](pictures/7-2.png)

In [16]:
from keras import layers
from keras import Input
from keras.models import Model

vocabulary_size = 50000
num_income_groups = 10

posts_inputs = Input(shape=(None,),dtype='int32',name='posts')
embedded_posts = layers.Embedding(256,vocabulary_size)(posts_inputs)

x = layers.Conv1D(128,5,activation='relu')(embedded_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256,5,activation='relu')(x)
x = layers.Conv1D(256,5,activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256,5,activation='relu')(x)
x = layers.Conv1D(256,5,activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128,activation='relu')(x)

age_prediction = layers.Dense(1,name='age')(x)
income_prediction = layers.Dense(num_income_groups,
                                 activation='softmax',
                                 name='income')(x)
gender_prediction = layers.Dense(1,activation='sigmoid',
                                 name='gender')(x)

model = Model(posts_inputs,
              [age_prediction,income_prediction,
               gender_prediction])